<a href="https://colab.research.google.com/github/sunday004/ai4all_project/blob/main/StockSentimentKaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("drive/MyDrive/Codes/stock-market-data.csv", encoding="ISO-8859-1")

In [ ]:
df.head()

Split dataset

In [4]:
train = df[df["Date"] < "20150101"]  # train data is before 2015
test = df[df["Date"] > "20141231"]   # Test data is after 2015

Preprocess

In [ ]:
# extract only the headlines
data = train.iloc[:, 2:27]

# Remove punctuations
data.replace("[^a-zA-Z]", " ", regex=True, inplace=True)

# Rename column names for easy access. TopX -> X
list1 = [i for i in range(25)]
new_index = [str(i) for i in list1]
data.columns = new_index
data.head()

In [ ]:
# convert headlines to lower case
for index in new_index:
    data[index] = data[index].str.lower()
data.head()

In [7]:
# Combine first row of data
' '.join(str(x) for x in data.iloc[0, 0:25])

'a  hindrance to operations   extracts from the leaked reports scorecard hughes  instant hit buoys blues jack gets his skates on at ice cold alex chaos as maracana builds up for united depleted leicester prevail as elliott spoils everton s party hungry spurs sense rich pickings gunners so wide of an easy target derby raise a glass to strupar s debut double southgate strikes  leeds pay the penalty hammers hand robson a youthful lesson saints party like it s      wear wolves have turned into lambs stump mike catches testy gough s taunt langer escapes to hit     flintoff injury piles on woe for england hunters threaten jospin with new battle of the somme kohl s successor drawn into scandal the difference between men and women sara denver  nurse turned solicitor diana s landmine crusade put tories in a panic yeltsin s resignation caught opposition flat footed russian roulette sold out recovering a title'

In [8]:
# combine headlines for all data
headlines = []
for row in range(len(data.index)):
    headlines.append(' '.join(str(x) for x in data.iloc[row, 0:25]))

In [9]:
headlines[0]

'a  hindrance to operations   extracts from the leaked reports scorecard hughes  instant hit buoys blues jack gets his skates on at ice cold alex chaos as maracana builds up for united depleted leicester prevail as elliott spoils everton s party hungry spurs sense rich pickings gunners so wide of an easy target derby raise a glass to strupar s debut double southgate strikes  leeds pay the penalty hammers hand robson a youthful lesson saints party like it s      wear wolves have turned into lambs stump mike catches testy gough s taunt langer escapes to hit     flintoff injury piles on woe for england hunters threaten jospin with new battle of the somme kohl s successor drawn into scandal the difference between men and women sara denver  nurse turned solicitor diana s landmine crusade put tories in a panic yeltsin s resignation caught opposition flat footed russian roulette sold out recovering a title'

In [10]:
# Convert all sentences into vectors using CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [11]:
# Now make a BAG of WORDS

# The countvectorizer
countvector = CountVectorizer(ngram_range=(2, 2))

# transform the sentences into vectors
traindataset = countvector.fit_transform(headlines)

In [12]:
traindataset.shape[0] == len(train)

True

In [13]:
# Code an RFC

# The classifier
randomclassifier = RandomForestClassifier(n_estimators=200, criterion="entropy")

# Classify the vectorized dataset with the Label they correspond to in train dataframe
randomclassifier.fit(traindataset, train["Label"])


RandomForestClassifier(criterion='entropy', n_estimators=200)

In [14]:
# Save random classifier model to a file
import pickle
with open("stock-sentiment-predict.pkl", "wb") as rfc_file:
    pickle.dump(randomclassifier, rfc_file)

In [15]:
# Save the random classifier model to a file using joblib
import joblib
joblib.dump(randomclassifier, "stock-sentiment-predict.joblib")

['stock-sentiment-predict.joblib']

In [ ]:
# Predict for the Test Dataset
test_transform = []
for row in range(len(test.index)):
    test_transform.append(' '.join(str(x) for x in test.iloc[row, 2:27]))

print(test_transform)

In [ ]:
# Remove punctuations and make lowercase in test_dataset;
# Test dataset python list is an array not an dataframe
# remove punctuations
# use re module
import re
# remove punctuations
test_datad = [re.sub(r"[^a-zA-Z]", " ", x).lower() for x in test_transform]

test_datad

In [28]:
# Vectorize the joined test dataset
test_dataset = countvector.transform(test_transform)

In [29]:
# Predict
predictions = randomclassifier.predict(test_dataset)

In [ ]:
predictions

In [31]:
# Import library to check accuracy
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [32]:
# Get confusion matrix
matrix = confusion_matrix(test["Label"], predictions)
print(matrix)

[[140  46]
 [  8 184]]


In [33]:
# Get accuracy score
score = accuracy_score(test["Label"], predictions)
print(score)

0.8571428571428571


In [34]:
# Generate model report
report = classification_report(test["Label"], predictions)
print(report)

              precision    recall  f1-score   support

           0       0.95      0.75      0.84       186
           1       0.80      0.96      0.87       192

    accuracy                           0.86       378
   macro avg       0.87      0.86      0.86       378
weighted avg       0.87      0.86      0.86       378



Check the meanings of the above - from the Week 10 lecture recording and slides

In [35]:
print(":)")

:)
